In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15.8 ms (started: 2021-08-25 15:02:45 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #1  Aug 25, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_developer_guides-.git (fetch)
origin	git@github.com:ustchope/keras_developer_guides-.git (push)
[main a24f968] 更新 #1  Aug 25, 2021
 1 file changed, 1382 insertions(+), 1 deletion(-)


To git@github.com:ustchope/keras_developer_guides-.git
   ce88496..a24f968  main -> main


time: 4.78 s (started: 2021-08-25 15:02:48 +08:00)


In [4]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.54 s (started: 2021-08-25 15:02:53 +08:00)


# 设置

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

time: 840 µs (started: 2021-08-25 15:05:14 +08:00)


# `Layer` 类：状态（权重）和一些计算的组合

Keras 中的核心抽象之一是 `Layer` 类。 层封装了状态（层的`weights`）和从输入到输出的转换（`call`，层的前向传递）。

这是一个密集连接层。 它有一个状态：变量 `w` 和 `b`。

In [6]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

time: 1.33 ms (started: 2021-08-23 18:11:04 +08:00)


您可以通过在一些张量输入上调用层来使用它，就像 Python 函数一样。

In [7]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[0.02051926 0.14575195 0.05476379 0.06521606]
 [0.02051926 0.14575195 0.05476379 0.06521606]], shape=(2, 4), dtype=float32)
time: 3.23 s (started: 2021-08-23 18:11:50 +08:00)


请注意，权重 `w` 和 `b` 在被设置为层属性后会被层自动跟踪：

In [8]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

time: 754 µs (started: 2021-08-23 18:14:19 +08:00)


请注意，您还可以使用更快的快捷方式为图层添加权重：add_weight() 方法：

In [9]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.04528809 -0.04564667 -0.01765442  0.0017395 ]
 [-0.04528809 -0.04564667 -0.01765442  0.0017395 ]], shape=(2, 4), dtype=float32)
time: 15.2 ms (started: 2021-08-23 18:15:01 +08:00)


# 层可以具有不可训练的权重

除了可训练的权重，您还可以向层添加不可训练的权重。 当您训练该层时，在反向传播期间不应考虑此类权重。

以下是添加和使用不可训练重量的方法：

In [11]:
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]
time: 18.8 ms (started: 2021-08-23 18:16:47 +08:00)


# 最佳实践：推迟权重创建，直到知道输入的形状

我们上面的 Linear 层采用 `input_dim` 参数，用于计算 `__init__()` 中权重 `w` 和 `b` 的形状：

In [12]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

time: 1.11 ms (started: 2021-08-23 18:17:54 +08:00)


在许多情况下，您可能事先不知道输入的大小，并且在实例化图层一段时间后，当该值已知时，您希望懒惰地创建权重。

在 Keras API 中，我们建议在层的 build(self,input_shape) 方法中创建层权重。 像这样：

In [13]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

time: 1.69 ms (started: 2021-08-23 18:18:35 +08:00)


层的 `__call__()` 方法将在第一次调用时自动运行 `build`。 你现在有一个懒惰的层，因此更容易使用：

# 层是可递归组合的

如果您将一个 Layer 实例指定为另一个 `Layer` 的属性，则外层将开始跟踪内层的权重。

我们建议在 `__init__()` 方法中创建这样的子层（因为子层通常有一个 `build` 方法，它们将在构建外层时构建）。

In [15]:
# 假设我们正在重用 Linear 类
# 使用我们上面定义的 `build` 方法。


class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6
time: 29.9 ms (started: 2021-08-23 18:21:36 +08:00)


# add_loss() 方法

在编写层的 `call()` 方法时，您可以创建稍后在编写训练循环时要使用的损失张量。 这可以通过调用 `self.add_loss(value)` 来实现：

In [16]:
# 创建活动正则化损失的层
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

time: 1.06 ms (started: 2021-08-23 18:24:13 +08:00)


这些损失（包括由任何内层创建的损失）可以通过 `layer.losses` 检索。 此属性在每次 `__call__()` 开始时重置到顶级层，因此 `layer.losses` 始终包含在上次前向传递期间创建的损失值。

In [17]:
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)


layer = OuterLayer()
assert len(layer.losses) == 0  # 没有损失，因为该层从未被调用过

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # 我们创建了一个损失值

# `layer.losses` 在每次 __call__ 开始时重置
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # 这是上面调用过程中产生的损失

time: 14.5 ms (started: 2021-08-23 18:26:11 +08:00)


此外，`loss`属性还包含为任何内层的权重创建的正则化损失：

In [18]:
class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=tf.keras.regularizers.l2(1e-3)
        )

    def call(self, inputs):
        return self.dense(inputs)


layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1, 1)))

# This is `1e-3 * sum(layer.dense.kernel ** 2)`,
# created by the `kernel_regularizer` above.
print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0018706928>]
time: 31.1 ms (started: 2021-08-23 18:28:14 +08:00)


在编写训练循环时要考虑这些损失，如下所示：

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Iterate over the batches of a dataset.
for x_batch_train, y_batch_train in train_dataset:
    with tf.GradientTape() as tape:
        logits = layer(x_batch_train)  # Logits for this minibatch
        # Loss value for this minibatch
        loss_value = loss_fn(y_batch_train, logits)
        # Add extra losses created during this forward pass:
        loss_value += sum(model.losses)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

有关编写训练循环的详细指南，请参阅从头开始编写训练循环的指南。

这些损失也可以与 fit() 无缝协作（它们会自动求和并添加到主要损失中，如果有的话）：

In [19]:
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# 如果在 `compile` 中传递了损失，则正则化损失被添加到它
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# 也可以在 `compile` 中不传递任何损失，
# 因为模型已经有一个损失要最小化，通过`add_loss`
# 在前传期间调用！
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

1/1 [==============================] - 0s 53ms/step - loss: 0.0365


time: 964 ms (started: 2021-08-23 18:32:40 +08:00)


# add_metric() 方法

与 `add_loss()` 类似，层也有一个 `add_metric()` 方法，用于在训练期间跟踪数量的移动平均值。

考虑以下层：“物流端点”层。 它将预测和目标作为输入，计算通过 `add_loss()` 跟踪的损失，并计算通过 `add_metric()` 跟踪的准确度标量。

In [22]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)

time: 1.85 ms (started: 2021-08-23 18:37:15 +08:00)


以这种方式跟踪的指标可以通过 `layer.metrics` 访问：

In [23]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))

layer.metrics: [<tensorflow.python.keras.metrics.BinaryAccuracy object at 0x7fc94c127d60>]
current accuracy value: 1.0
time: 55 ms (started: 2021-08-23 18:37:48 +08:00)


就像 `add_loss()` 一样，这些指标由 `fit()` 跟踪：

In [24]:
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

1/1 [==============================] - 0s 276ms/step - loss: 1.1473 - binary_accuracy: 0.0000e+00


time: 475 ms (started: 2021-08-23 18:39:02 +08:00)


# 您可以选择在图层上启用序列化

如果您需要将自定义层作为功能模型的一部分进行序列化，您可以选择实现 get_config() 方法：

In [25]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"units": self.units}


# Now you can recreate the layer from its config:
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'units': 64}
time: 3.77 ms (started: 2021-08-23 18:45:17 +08:00)


请注意，基础 `Layer` 类的 `__init__()` 方法采用一些关键字参数，特别是名称和数据类型。 将这些参数传递给 `__init__()` 中的父类并将它们包含在层配置中是一种很好的做法：

In [26]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config


layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'name': 'linear_7', 'trainable': True, 'dtype': 'float32', 'units': 64}
time: 3.82 ms (started: 2021-08-23 18:47:13 +08:00)


如果在从其配置反序列化层时需要更大的灵活性，您还可以覆盖 `from_config()` 类方法。 这是 `from_config()` 的基本实现：

In [ ]:
def from_config(cls, config):
    return cls(**config)

要了解有关序列化和保存的更多信息，请参阅保存和序列化模型的完整指南。

# call() 方法中的优先训练参数

某些层，尤其是 `BatchNormalization` 层和 `Dropout` 层，在训练和推理过程中具有不同的行为。 对于此类层，标准做法是在 `call()` 方法中公开训练（布尔值）参数。

通过在 `call()` 中公开此参数，您可以启用内置的训练和评估循环（例如 `fit()`）以在训练和推理中正确使用该层。

In [27]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

time: 1.19 ms (started: 2021-08-23 18:49:24 +08:00)


# call() 方法中的优先掩码参数

`call()` 支持的另一个特权参数是`mask`参数。

您会在所有 Keras RNN 层中找到它。 掩码是一个布尔张量（输入中每个时间步长一个布尔值），用于在处理时间序列数据时跳过某些输入时间步长。

当前一层生成掩码时，Keras 将自动将正确的`mask`参数传递给支持它的层的 `__call__()` 。 生成`mask`的层是配置`mask_zero=True`的`Embedding`层和`Masking`层。

要了解有关掩码以及如何编写启用遮罩的图层的更多信息，请查看指南“了解填充和掩码”。

# 模型类

通常，您将使用 `Layer` 类来定义内部计算块，并使用 `Model` 类来定义外部模型——您将训练的对象。

例如，在 ResNet50 模型中，您将有多个 ResNet 块子类化层，以及一个包含整个 ResNet50 网络的模型。

`Model` 类与 `Layer` 具有相同的 API，但有以下区别：
* 它公开了内置的训练、评估和预测循环（`model.fit()`、`model.evaluate()`、`model.predict()`）。
* 它通过 `model.layers` 属性公开其内部层的列表。
* 它公开了保存和序列化 API（`save()`、`save_weights()`...）

实际上，`Layer` 类对应于我们在文献中所说的“层”（如“卷积层”或“循环层”）或“块”（如“ResNet 块”或“初始块”） ）。

同时，`Model` 类对应于文献中所谓的“模型”（如“深度学习模型”）或“网络”（如“深度神经网络”）。

因此，如果您想知道，“我应该使用 `Layer` 类还是 `Model` 类？”，请问自己：我是否需要对其调用 `fit()` ？ 我需要调用 `save()` 吗？ 如果是这样，请选择模型。 如果不是（因为您的类只是更大系统中的一个块，或者因为您正在编写培训和保存代码），请使用 `Layer`。

例如，我们可以采用上面的 mini-resnet 示例，并使用它来构建一个模型，我们可以使用 `fit()` 训练该模型，并且可以使用 `save_weights()` 进行保存：

In [ ]:
class ResNet(tf.keras.Model):

    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)


resnet = ResNet()
dataset = ...
resnet.fit(dataset, epochs=10)
resnet.save(filepath)

# 把它们放在一起：一个端到端的例子

以下是您到目前为止所学到的：
* 一个层封装了一个状态（在 `__init__()` 或 `build()` 中创建）和一些计算（在 `call()` 中定义）。
* 层可以递归嵌套以创建新的、更大的计算块。
* 层可以通过 `add_loss()` 和 `add_metric()` 创建和跟踪损失（通常是正则化损失）以及度量
* 外层容器，你要训练的东西，是一个模型。 模型就像一个层，但增加了训练和序列化实用程序。

让我们把所有这些东西放在一个端到端的例子中：我们将实现一个变分自动编码器 (VAE)。 我们将在 MNIST 数字上训练它。

我们的 VAE 将是 `Model` 的子类，构建为子类 `Layer` 的嵌套层组合。 它将具有正则化损失（KL 散度）。

In [9]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
#         batch, dim = z_mean.shape
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
#         epsilon = tf.random_normal(shape=(batch,dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z
    
class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)
    
class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

time: 3.3 ms (started: 2021-08-25 15:05:27 +08:00)


让我们在 MNIST 上编写一个简单的训练循环：

In [11]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = 0.3232
step 100: mean loss = 0.1249
step 200: mean loss = 0.0989
step 300: mean loss = 0.0890
step 400: mean loss = 0.0841
step 500: mean loss = 0.0808
step 600: mean loss = 0.0787
step 700: mean loss = 0.0771
step 800: mean loss = 0.0759
step 900: mean loss = 0.0749
Start of epoch 1
step 0: mean loss = 0.0746
step 100: mean loss = 0.0740
step 200: mean loss = 0.0735
step 300: mean loss = 0.0730
step 400: mean loss = 0.0727
step 500: mean loss = 0.0723
step 600: mean loss = 0.0720
step 700: mean loss = 0.0717
step 800: mean loss = 0.0714
step 900: mean loss = 0.0712
time: 29.7 s (started: 2021-08-25 15:07:26 +08:00)


请注意，由于 VAE 是模型的子类，它具有内置的训练循环。 所以你也可以这样训练它：

In [50]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

Epoch 1/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0747
Epoch 2/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0676


time: 18.4 s (started: 2021-08-23 19:19:42 +08:00)


# 超越面向对象的开发：函数式 API

这个例子对你来说是不是太多面向对象的开发？ 您还可以使用 Functional API 构建模型。 重要的是，选择一种或另一种风格并不妨碍您利用以另一种风格编写的组件：您始终可以混合搭配。

例如，下面的函数式 API 示例重用了我们在上面的示例中定义的相同采样层：

In [12]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 10s 10ms/step - loss: 0.0745
Epoch 2/3
938/938 [==============================] - 9s 10ms/step - loss: 0.0676
Epoch 3/3
938/938 [==============================] - 9s 10ms/step - loss: 0.0675


time: 29.4 s (started: 2021-08-25 15:09:35 +08:00)


有关更多信息，请务必阅读 Functional API 指南。